In [1]:
import numpy as np
import pysocialforce as psf

DEBUG:[__init__.py:350              wrapper() ] matplotlib data path: c:\Users\koenw\AppData\Local\Programs\Python\Python312\Lib\site-packages\matplotlib\mpl-data
DEBUG:[__init__.py:350              wrapper() ] CONFIGDIR=C:\Users\koenw\.matplotlib
DEBUG:[__init__.py:1511             <module>() ] interactive is False
DEBUG:[__init__.py:1512             <module>() ] platform is win32
DEBUG:[__init__.py:350              wrapper() ] CACHEDIR=C:\Users\koenw\.matplotlib
DEBUG:[font_manager.py:1574    _load_fontmanager() ] Using fontManager instance from C:\Users\koenw\.matplotlib\fontlist-v330.json


In [52]:
# Generate random starting positions for 10 agents
num_agents = 10
np.random.seed(42)  # Set seed for reproducibility
initial_positions = np.random.uniform(low=[-5, -5], high=[10, 10], size=(num_agents, 2))

# Define goal position
goal_position = np.array([2, 12])
goal_threshold = 0.5  # Distance threshold to consider an agent as having reached the goal

# Initialize velocities towards the goal
def initialize_velocity(position, goal):
    direction = goal - position
    norm = np.linalg.norm(direction)
    if norm == 0:
        return np.array([0, 0])
    return direction / norm  # Unit vector towards the goal

# Create initial state with velocities towards the goal
initial_state = np.array([
    [x, y, *initialize_velocity(np.array([x, y]), goal_position), goal_position[0], goal_position[1]]
    for x, y in initial_positions
])

# Initialize the simulator
sim = psf.Simulator(
    initial_state, groups=None, obstacles=None, config_file="loes.toml"
)

# Counter for agents that have reached the goal
agents_reached_goal = 0
agents_already_counted = set()  # Set to track counted agents to avoid double counting since removing agents from the simulator is not yet working

# Function to check if an agent has reached the goal
def check_reached_goal(agent_position, goal, threshold):
    distance = np.linalg.norm(agent_position - goal)
    return distance <= threshold

# Perform simulation steps and remove agents that reach the goal
for step in range(50):
    sim.step(n=1)
    ped_states, group_states = sim.get_states()  # Access the current state of agents

    # Extract the relevant frame (assuming we need the latest frame, index 0 for instance)
    current_state = ped_states[-1]  # Use the last frame for the current state

    new_ped_states = []
    for i, agent_state in enumerate(current_state):
        agent_position = agent_state[:2]  # Ensure agent_position is an array of shape (2,)

        if check_reached_goal(agent_position, goal_position, goal_threshold):
            if i not in agents_already_counted:
                agents_reached_goal += 1
                agents_already_counted.add(i)
        else:
            new_ped_states.append(agent_state)
    
    # Update the simulator's state directly with agents that haven't reached the goal
    sim.ped_states = [np.array(new_ped_states)] if new_ped_states else []

    if len(new_ped_states) == 0:
        break  # Exit the loop if all agents have reached the goal

print(f"Number of agents that reached the goal: {agents_reached_goal}")

Number of agents that reached the goal: 3


In [53]:
# Visualize the scene
with psf.plot.SceneVisualizer(sim, "one_goal") as sv:
    sv.animate()

INFO:[plot.py:131            __enter__() ] Start plotting.
INFO:[plot.py:162             __exit__() ] Plotting ends.
INFO:[plot.py:166             __exit__() ] Saving animation as one_goal.gif
DEBUG:[animation.py:669          isAvailable() ] ImageMagick unavailable due to: Failed to find an ImageMagick installation
INFO:[animation.py:1060                 save() ] Animation.save using <class 'matplotlib.animation.PillowWriter'>
